In [1]:
############### Set working directory to parent directory
import os,  requests, sys, re, urllib.request, pandas as pd
from bs4 import BeautifulSoup

os.getcwd()
# os.chdir('F:\\github\\narrative_conservatism\\code')

'F:\\github\\narrative_conservatism\\code'

In [2]:
#################### Access all fillings through SEC master index #################################
####### indexes downloaded using python-edgar: https://github.com/edouardswiac/python-edgar #######
#### open terminal, and run the following lines:
#### cd F:\github\python-edgar-master (switch dir to where the run.py script is located)
#### python run.py -y 1993 -d edgar_idx (downloading all quarterly master index from 1993 into folder edgar_idx)
#### cd F:\github\python-edgar-master\edgar-idx (switch dir to where the downloaded indexes are located)
#### cat *.tsv > master.tsv (stitch all quarterly indexes into one master index)
#### du -h master.tsv (inspect how large the master index file is)

obj_type = '10-Q'

doc_url = list()
# create an index of downloaded local quarterly master indexes
index_input = list()
for subdir, dirs, files in os.walk("F:\github\python-edgar-master\edgar-idx"):
    for file in files:
        index_input.append(os.path.join(subdir, file))

# read each index file, select rows with matched file type, and store matched doc_links
for filenameTSV in index_input:
    tsv_read = pd.read_csv(filenameTSV, sep='|', header=None, encoding = "utf-8")
    tsv_read.columns = ['1', '2', '3', '4', '5', '6']
    
    # select the rows with filetype equal to predefined type
    tsv_type = tsv_read.loc[tsv_read['3'] == obj_type]
    doc_link = tsv_type['6'].values.tolist()
    doc_link = ['https://www.sec.gov/Archives/' + w for w in doc_link]
    for doc in doc_link:
        doc_url.append(doc)
        
len(doc_url)

594017

In [3]:
# #################### Access all fillings through SEC search engine ####################################
# ################## NOT RECOMMENDED AT ALL #############################################################
# cik = '0000051143'
# obj_type = '8-K'
# number of documents listed per page
# count = '100'
# # index of first document listed in the current page
# start = '0'
# # find filings prior to the date 2016y01m01d
# dateb = ''

# # Obtain url for intial search result page
# base_url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type={}&dateb={}&start={}&count={}"
# init_url = base_url.format(cik, obj_type, dateb, start, count)

# # define a function that takes the input url and returns next search page url
# def get_next_url(input_url):
#     edgar_resp = requests.get(input_url)
#     edgar_str = edgar_resp.text
#     soup = BeautifulSoup(edgar_str, 'html.parser')

#     div_tag = soup.find('div', style='margin-top: 5px; margin-bottom: 5px;')
#     button = div_tag.find('td', style='text-align: right;')
#     fbutton = button.find_all('input')[0]['value']
#     if re.findall(r'Next', fbutton) == ['Next']:
#         next_url = button.find_all('input')[0]['onclick'][:-1]
#     elif len(button.find_all('input')) == 2:
#         next_url = button.find_all('input')[1]['onclick'][:-1]
#     else:
#         next_url = 'NA'
        
#     next_url = next_url.replace('parent.location=\'', 'https://www.sec.gov')
#     return next_url

# # create a search result page url list
# search_url = [init_url]

# while get_next_url(init_url) != 'NA':
#     search_url.append(get_next_url(init_url))
#     init_url = get_next_url(init_url)
    
# ############### Create a document link list of a given CIK and file type
# doc_link = list()

# for url in search_url:
#     edgar_resp = requests.get(url)
#     edgar_str = edgar_resp.text
#     soup = BeautifulSoup(edgar_str, 'html.parser')
#     table_tag = soup.find('table', class_='tableFile2')
#     rows = table_tag.find_all('tr')

#     for row in rows[1:]:
#         cells = row.find_all('td')
#         doc_link.append('https://www.sec.gov' + cells[1].a['href'])
        
# len(doc_link)

In [16]:
################## Save the HTML/TXT website urls from doc_url to raw data folder
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}
web_url = list()

for doc in doc_url[-200:]:
    doc_resp = requests.get(doc, headers=headers)
    doc_str = doc_resp.text
    soup = BeautifulSoup(doc_str, 'html.parser')
    
    table_tag = soup.find('table', class_='tableFile', summary='Document Format Files')
    rows = table_tag.find_all('tr')
    cell_html = rows[1].find_all('td')
    html = cell_html[2].a['href'].replace('ix?doc=/', '')
    cell_txt = rows[-1].find_all('td')
    txt = cell_txt[2].a['href']
    
    if html.endswith("htm"):
        web_url.append('https://www.sec.gov' + html)
    else:
        web_url.append('https://www.sec.gov' + txt)

## Save to local index
with open("..\\filings\\web_url_index.txt", "w") as f:
    for s in web_url:
        f.write(s +"\n")
        
# web_url[-20:]

In [4]:
############### Extract file identification info from doc_url
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}
accnum = list()
fd = list()
rp = list()
name = list()
cik = list()
sic = list()
filetype = list()
fye = list()
state = list()
bazip = list()
item8k = list()

for doc in doc_url[-200:]:
    doc_resp = requests.get(doc, headers=headers)
    doc_str = doc_resp.text
    soup = BeautifulSoup(doc_str, 'html.parser')
# Save the SEC accession number (accnum)
    div_tag = soup.find('div', id='formHeader')
    secnum = div_tag.find('div', id='secNum')
    a = secnum.get_text().split()[3]
    accnum.append(a)
    
# Save the Filing Date and Reporting Period
    div_tag = soup.find('div', class_='formContent')
    dates = div_tag.find_all('div', class_='info')
    # Filing Date
    a = dates[0].get_text()
    fd.append(a)
    # Reporting Period
    b = dates[3].get_text()
    rp.append(b)
    # For 8K files, Save item info
    if obj_type == '8-K':
        c = dates[4].get_text()
        clist = re.findall(r'\d.\d\d', c)
        if clist != []:
            c = ', '.join(clist)
            item8k.append(c)
        else:
            clist = re.findall(r'\d', c)
            c = ', '.join(clist)
            item8k.append(c)
    else :
        c = 'NA'
        item8k.append(c)
        
# Save the Company name and CIK
    div_tag = soup.find('div', class_='companyInfo')
    comname = div_tag.find('span', class_='companyName')
    # Company Name
    a = comname.get_text().split("\n")[0].replace(' (Filer)', '')
    name.append(a)
    # CIK
    b = comname.get_text().split("\n")[1].replace('CIK: ', '').replace(' (see all company filings)', '')
    cik.append(b)
    
# Save Business Address ZIP 
    div_tag = soup.find_all('div', class_='mailer')[1].find_all('span', class_='mailerAddress')[1]
    ba = div_tag.get_text()
    alist = re.findall(r'\d\d\d\d\d', ba)
    if alist == []:
        div_tag = soup.find_all('div', class_='mailer')[1].find_all('span', class_='mailerAddress')[2]
        ba = div_tag.get_text()
        alist = re.findall(r'\d\d\d\d\d', ba)
    a = ', '.join(alist)
    bazip.append(a)
    
# Save SIC, File Type, Fiscal Year End and State of Incorporation
    div_tag = soup.find('div', class_='companyInfo')
    filinginfo = div_tag.find('p', class_='identInfo')
    # SIC
#   a = filinginfo.get_text().split("|")[5].split("SIC")[1].split()[1]
#   sic.append(a)
    # File Type
#   b = filinginfo.get_text().split("|")[2].split("Type")[1].split(":")[1]
#   filetype.append(b)
    # Fiscal Year End
    c = filinginfo.get_text().split("|")[2].split("Type")[0].split(":")[1]
    fye.append(c)
    # State
    d = filinginfo.get_text().split("|")[1].split(":")[1]
    state.append(d)

## Save accnum to local index
with open("..\\filings\\accnum_index.txt", "w") as f:
    for s in accnum:
        f.write(s +"\n")

In [6]:
############### Create Data Frame
d = {'cik': cik, 'name': name, 'fd': fd, 'rp': rp, 'filetype': obj_type, 'item8k': item8k, 'accnum': accnum, 'bazip': bazip}
id_data = pd.DataFrame(data=d)
id_data

,cik,name,fd,rp,filetype,item8k,accnum,bazip
0,0000733269,"LiveRamp Holdings, Inc.",2020-02-05,2019-12-31,10-Q,NA,0000733269-20-000012,94104
1,0000737758,TORO CO,2020-03-05,2020-01-31,10-Q,NA,0000737758-20-000007,55420
2,0000074046,Oil-Dri Corp of America,2020-03-05,2020-01-31,10-Q,NA,0000074046-20-000009,60611
3,0000743316,MAXIM INTEGRATED PRODUCTS INC,2020-01-29,2019-12-28,10-Q,NA,0000743316-20-000008,95134
4,0000743988,XILINX INC,2020-01-28,2019-12-28,10-Q,NA,0000743988-20-000009,95124
...,...,...,...,...,...,...,...,...
195,0000096699,TECHNICAL COMMUNICATIONS CORP,2020-02-11,2019-12-28,10-Q,NA,0001171843-20-000876,01742
196,0000096793,SUNLINK HEALTH SYSTEMS INC,2020-02-13,2019-12-31,10-Q,NA,0001564590-20-004619,30339
197,0000096885,TEL INSTRUMENT ELECTRONICS CORP,2020-02-13,2019-12-31,10-Q,NA,0001185185-20-000178,07073
198,0000098338,TSR INC,2020-01-13,2019-11-30,10-Q,NA,0001213900-20-000888,11788


In [11]:
# ############### Download HTML into TXT files (NOT RECOMMANDED DUE TO LARGE FILE SIZE)
# for link in web_url:
#     if os.path.exists('..\\filings\\raw\\'+str(accnum[web_url.index(link)])+'.txt') == False:
#         urllib.request.urlretrieve(link, '..\\filings\\raw\\'+str(accnum[web_url.index(link)])+'.txt')